In [ ]:
'''
Author: skyous 1019364238@qq.com
Date: 2024-01-18 10:47:09
LastEditors: skyous 1019364238@qq.com
LastEditTime: 2024-01-18 10:57:40
FilePath: /prostate158-main/test.ipynb
Description: 

Copyright (c) 2024 by ${git_name_email}, All Rights Reserved. 
'''

# Segmentation Sest Example
> Train a U-Net for pixelwise segmentation of the prostate

In [1]:
import monai
import ignite
import os


from prostate158.utils import load_config
from prostate158.train import SegmentationTrainer
# from prostate158.report import ReportGenerator
# from prostate158.viewer import ListViewer
from prostate158.data import segmentation_dataloaders
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All parameters needed for training and evaluation are set in `anatomy.yaml` file. 

In [2]:
config_path = 'config/prostate158/anatomy_mixunetr_non.yaml'
# model_pth_path = '/home/data1/skyous/prostatex_crop_log/swinunetr/models/network_anatomyprostatex_crop20240402_1551_swinunetr_48_1_key_metric=0.8420.pt'
model_pth_path = '/home/data1/skyous/prostate158_log/mixunetr/models/network_anatomyprostate15820240417_1043_mixunetr_non_1_key_metric=0.8245.pt'
config = load_config(config_path) # change to 'tumor.yaml' for tumor segmentation
monai.utils.set_determinism(seed=config.seed)
config.data.valid_csv = config.data.test_csv 
# config.data.valid_csv = '/home/data1/skyous/prostate158/prostate158_test/test.csv'
print(config.data.valid_csv)

/home/data1/skyous/prostate158/test.csv


In [3]:
test_loader=segmentation_dataloaders(
            config=config, 
            train=False, 
            valid=False, 
            test=True
        )

trainer=SegmentationTrainer(
    progress_bar=True, 
    early_stopping = True, 
    metrics = ["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics = True,
    config=config
)
trainer.evaluate(checkpoint=model_pth_path)

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.handlers.stats_handler StatsHandler.__init__:name: Current default value of argument `name=None` has been deprecated since version 1.1. It will b

metrics saved to /home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240418_0858_mixunetr_non_1/output


/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0953_mixunetr_1/output
/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0957_mixunetr_1/output
./prostatex_crop_log/UXNET/anatomyprostatex_crop20240321_1043_UXNET_1/output
./prostatex_crop_log/nnformer/anatomyprostatex_crop20240321_1047_nnformer_1/output
/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1053_mixunetr_1/output
/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1057_mixunetr_1/output
/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1132_3dunet_1/output
/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1136_3dunet_1/output
/home/data1/skyous/prostatex_crop_log/unetr/anatomyprostatex_crop20240321_2041_unetr_1/output

# 计算平均的dice系数

In [4]:
import pandas as pd
import os

output_dir = '/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240418_0858_mixunetr_non_1/output'
# 读取CSV文件
DSC_data = pd.read_csv(os.path.join(output_dir, 'MeanDice_raw.csv'))
HD_data = pd.read_csv(os.path.join(output_dir, 'HausdorffDistance_raw.csv'))
SD_data = pd.read_csv(os.path.join(output_dir, 'SurfaceDistance_raw.csv'))

output_ = '|'
for data in [DSC_data, HD_data, SD_data]:
    # 获取某一列的数值

    for item in ['class0', 'class1', 'mean']:
        column_values = data[item]

        # 计算平均值
        average = column_values.mean()
        if data is DSC_data:
            average = round(average, 4)
        else:
            average = round(average, 2)
        print(item, average)
        output_ += str(average) + '|'

output_ += output_dir + '|'
print(output_)

class0 0.8839
class1 0.7651
mean 0.8245
class0 15.87
class1 19.89
mean 17.88
class0 1.97
class1 2.07
mean 2.02
|0.8839|0.7651|0.8245|15.87|19.89|17.88|1.97|2.07|2.02|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240418_0858_mixunetr_non_1/output|


# prostatex
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|swinunetr_48|0.865|0.7999|0.8324|19.83|19.66|19.74|2.35|2.03|2.19|/home/data1/skyous/prostatex_crop_log/swinunetr/anatomyprostatex_crop20240321_1038_swinunetr_48_1/output|
|UXNET|0.86|0.7983|0.8292|19.95|18.21|19.08|2.31|2.05|2.18|./prostatex_crop_log/UXNET/anatomyprostatex_crop20240321_1043_UXNET_1/output|
|nnformer|0.8366|0.7341|0.7853|19.55|22.05|20.8|2.86|3.32|3.09|./prostatex_crop_log/nnformer/anatomyprostatex_crop20240321_1047_nnformer_1/output|
|mixunetr_dice|0.8669|0.7987|0.8328|19.45|18.99|19.22|2.24|2.01|2.13|/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1053_mixunetr_1/output|
|mixunetr_bdou|0.8611|0.7983|0.8297|19.77|18.84|19.31|2.49|2.1|2.29|/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1057_mixunetr_1/output|
|3dResUNet|0.865|0.7918|0.8284|17.55|18.94|18.25|2.14|1.99|2.06|/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1132_3dunet_1/output|
|3dUNet|0.8541|0.7724|0.8132|18.98|18.41|18.7|2.61|2.24|2.43|/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1136_3dunet_1/output|
|unetr_42|0.8586|0.7873|0.823|19.93|18.72|19.32|2.61|2.11|2.36|/home/data1/skyous/prostatex_crop_log/unetr/anatomyprostatex_crop20240321_2041_unetr_1/output|
|mixunetr_q|0.8655|0.8027|0.8341|19.4|18.44|18.92|2.16|1.89|2.03|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240323_0846_mixunetr_q_1/output|
|mixunetr_k_bdou|0.8608|0.7979|0.8294|19.56|18.55|19.05|2.46|1.94|2.2|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240326_1427_mixunetr_k_1/output|
|mixunetr_qkv|0.8622|0.7976|0.8299|19.72|18.08|18.9|2.23|1.87|2.05|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240329_1120_mixunetr_qkv_1/output|
|mixunetr_qkv_trunc_normal|0.8625|0.8012|0.8318|19.4|19.47|19.43|2.27|1.92|2.09|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240331_0959_mixunetr_qkv_1/output|


# prostatex_seed1024
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|3dunet|0.8569|0.7717|0.8143|18.71|18.18|18.45|2.53|2.32|2.43|/home/data/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240402_1452_3dunet_1/output|
|unetr|0.8587|0.7853|0.822|20.23|19.24|19.73|2.67|2.28|2.48|/home/data1/skyous/prostatex_crop_log/unetr/anatomyprostatex_crop20240402_0803_unetr_1/output|
|swinunetr_1024|0.8575|0.7959|0.8267|19.75|18.88|19.32|2.62|1.94|2.28|/home/data1/skyous/prostatex_crop_log/swinunetr/anatomyprostatex_crop20240403_1424_swinunetr_48_1/output|
|nnformer|0.8354|0.7312|0.7833|20.09|22.29|21.19|2.9|3.51|3.2|/home/data/skyous/prostatex_crop_log/nnformer/anatomyprostatex_crop20240402_1455_nnformer_1/output|
|uxnet|0.8624|0.7997|0.831|19.6|20.35|19.98|2.37|2.02|2.2|/home/data/skyous/prostatex_crop_log/UXNET/anatomyprostatex_crop20240403_1431_UXNET_1/output|
|mixunetr_qkv_seed|0.8657|0.7997|0.8327|18.86|19.65|19.26|2.22|2.05|2.13|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240330_1211_mixunetr_qkv_1/output|






# prostate158
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|swinunetr_48| 0.8780|0.7555|0.8168|17.79|22.69|20.24|2.26|1.66|1.96|/home/data1/skyous/prostate158_log/swinunetr/anatomyprostate15820240320_2301_swinunetr_48_1/output|
|mixunetr_dice_2222|0.8837|0.7739|0.8288|17.93|21.93|19.93|2.04|1.66|1.85|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0939_mixunetr_1/output|
|mixunetr_bdouloss_2222|0.8833|0.7792|0.8312|16.84|20.58|18.71|2.04|1.67|1.86|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0945_mixunetr_1/output|
|mixunetr_bdouloss_1262|0.8831|0.775|0.829|17.02|20.96|18.99|2.11|1.64|1.88|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240321_2114_mixunetr_1/output|
|mixunetr_bdouloss_1266|0.8842|0.7702|0.8272|17.02|22.27|19.64|2.07|1.64|1.86|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0953_mixunetr_1/output|
|mixunetr_bdouloss_2242|0.8826|0.7773|0.8299|17.4|21.7|19.55|2.05|1.63|1.84|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0957_mixunetr_1/output|
|unetr_48|0.8443|0.6929|0.7686|22.46|26.8|24.63|3.16|2.48|2.82|/home/data/skyous/prostate158_log/unetr/anatomyprostate15820240321_2058_unetr_1/output|
|UXNET|0.8794|0.7686|0.824|17.97|22.42|20.2|2.15|1.67|1.91|/home/data/skyous/prostate158_log/UXNET/anatomyprostate15820240321_2101_UXNET_1/output|
|nnformer|0.8473|0.6876|0.7675|18.54|23.27|20.9|2.77|2.32|2.55|/home/data/skyous/prostate158_log/nnformer/anatomyprostate15820240321_2104_nnformer_1/output|
|mixunetr_bdouloss_2442|0.8711|0.7478|0.8094|18.45|23.42|20.94|2.19|1.7|1.94|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240321_2054_mixunetr_1/output|
|3dunet|0.8582|0.6996|0.7789|21.05|25.95|23.5|2.53|2.11|2.32|/home/data/skyous/prostate158_log/3dunet/anatomyprostate15820240321_2110_3dunet_2/output|
|3dresunet|0.8801|0.7624|0.8212|18.2|23.17|20.68|2.1|1.79|1.95|/home/data/skyous/prostate_log/3dunet/anatomyprostate15820240321_2111_3dunet_1/output|
|mixunetr_q|0.8829|0.7665|0.8247|15.72|19.88|17.8|1.9|2.0|1.95|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240322_0830_mixunetr_q_1/output|
|mixunetr_k|0.8836|0.7737|0.8286|17.52|22.02|19.77|2.06|1.66|1.86|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240323_0839_mixunetr_k_1/output|

|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_v|0.8837|0.7739|0.8288|17.93|21.93|19.93|2.04|1.66|1.85|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0939_mixunetr_1/output|
|mixunetr_q|0.8829|0.7665|0.8247|15.72|19.88|17.8|1.9|2.0|1.95|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240322_0830_mixunetr_q_1/output|
|mixunetr_k|0.8836|0.7737|0.8286|17.52|22.02|19.77|2.06|1.66|1.86|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240323_0839_mixunetr_k_1/output|

|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_bdouloss_1262|0.8831|0.775|0.829|17.02|20.96|18.99|2.11|1.64|1.88|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240321_2114_mixunetr_1/output|
|mixunetr_bdouloss_1266|0.8842|0.7702|0.8272|17.02|22.27|19.64|2.07|1.64|1.86|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0953_mixunetr_1/output|
|mixunetr_bdouloss_2222|0.8833|0.7792|0.8312|16.84|20.58|18.71|2.04|1.67|1.86|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0945_mixunetr_1/output|
|mixunetr_bdouloss_2242|0.8826|0.7773|0.8299|17.4|21.7|19.55|2.05|1.63|1.84|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0957_mixunetr_1/output|
|mixunetr_bdouloss_2442|0.8711|0.7478|0.8094|18.45|23.42|20.94|2.19|1.7|1.94|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240321_2054_mixunetr_1/output|

class0 0.8831
class1 0.775
mean 0.829
class0 17.02
class1 20.96
mean 18.99
class0 2.11
class1 1.64
mean 1.88

# ablation1 the number of blocks
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_dice_2222|0.8837|0.7739|0.8288|17.93|21.93|19.93|2.04|1.66|1.85|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0939_mixunetr_1/output|
|mixunetr_dice_2242|0.8806|0.7698|0.8252|18.23|22.08|20.15|2.15|1.66|1.91|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240325_0906_mixunetr_2/output|
|mixunetr_dice_2244|0.8852|0.7754|0.8303|17.59|21.09|19.34|2.04|1.71|1.87|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240325_0908_mixunetr_1/output|
|mixunetr_dice_2262|0.8816|0.7682|0.8249|17.99|22.55|20.27|2.09|1.65|1.87|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240325_0912_mixunetr_1/output|
|mixunetr_dice_2266|0.8832|0.7677|0.8255|18.3|22.07|20.19|2.11|1.64|1.87|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240325_0912_mixunetr_1/output|

# ablation1 the number of blocks for mixunetr_qkv
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_qkv_1111|0.8779|0.7669|0.8224|18.17|21.32|19.75|2.12|1.68|1.9|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240329_0828_mixunetr_qkv_1/output|
|mixunetr_qkv_1121|0.8815|0.7642|0.8228|18.17|22.68|20.43|2.14|1.63|1.89|/home/data1/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240329_0854_mixunetr_qkv_1/output|
|mixunetr_qkv_1122|0.8806|0.7669|0.8237|17.84|21.47|19.66|2.21|1.73|1.97|/home/data/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240401_0937_mixunetr_qkv_1/output|
|mixunetr_qkv_2222|0.8844|0.778|0.8312|17.69|22.09|19.89|2.04|1.68|1.86|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240328_1120_mixunetr_qkv_1/output|
|mixunetr_qkv_2242|0.8793|0.7636|0.8214|18.66|22.44|20.55|2.14|1.69|1.92|/home/data1/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240331_1006_mixunetr_qkv_1/output|
|mixunetr_qkv_2244|0.8784|0.7628|0.8206|17.89|22.05|19.97|2.27|1.77|2.02|/home/data1/skyous/prostate158_log/mixunetr_ablation/anatomyprostate15820240331_1009_mixunetr_qkv_1/output|

# abaltion2 the qkv of channel interaction
## prostate158
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_non|0.8839|0.7651|0.8245|15.87|19.89|17.88|1.97|2.07|2.02|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240418_0858_mixunetr_non_1/output|
|mixunetr_q|0.8829|0.7665|0.8247|15.72|19.88|17.8|1.9|2.0|1.95|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240322_0830_mixunetr_q_1/output|
|mixunetr_k|0.8836|0.7737|0.8286|17.52|22.02|19.77|2.06|1.66|1.86|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240323_0839_mixunetr_k_1/output|
|mixunetr_v|0.8837|0.7739|0.8288|17.93|21.93|19.93|2.04|1.66|1.85|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240321_0939_mixunetr_1/output|
|mixunetr_qk|0.8826|0.7699|0.8262|17.29|22.83|20.06|2.1|1.66|1.88|/home/data1/skyous/prostate158_log/mixunetr/anatomyprostate15820240328_0907_mixunetr_qk_1/output|
|mixunetr_kv|0.8846|0.774|0.8293|17.4|21.93|19.67|2.06|1.61|1.84|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240328_1115_mixunetr_kv_1/output|
|mixunetr_qv|0.8844|0.7766|0.8305|17.49|21.84|19.66|2.04|1.65|1.84|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240328_1118_mixunetr_qv_1/output|
|mixunetr_qkv|0.8844|0.778|0.8312|17.69|22.09|19.89|2.04|1.68|1.86|/home/data/skyous/prostate158_log/mixunetr/anatomyprostate15820240328_1120_mixunetr_qkv_1/output|

## prostatex
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|mixunetr_q|0.8655|0.8027|0.8341|19.4|18.44|18.92|2.16|1.89|2.03|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240325_0923_mixunetr_q_1/output|
|mixunetr_k|0.867|0.8021|0.8345|19.47|18.22|18.85|2.19|1.92|2.06|/home/data/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240325_0924_mixunetr_k_1/output|
|mixunetr_v|0.8669|0.7987|0.8328|19.45|18.99|19.22|2.24|2.01|2.13|/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1053_mixunetr_1/output|

In [9]:

import pandas as pd

# 读取CSV文件
data = pd.read_csv('/home/data1/skyous/prostate158_log/swinunetr/anatomyprostate15820240320_2301_swinunetr_48_1/output/val_mean_dice_raw.csv')

# 获取某一列的数值
column_values = data['class1'] # class0, class1, mean

# 计算平均值
average = column_values.mean()

print("平均值:", average)





class0 0.8779660694736843
class1 0.7555469294736842
mean 0.8167564947368422
class0 17.788902605263154
class1 22.69254147368421
mean 20.240721999999995
class0 2.2560492105263155
class1 1.6599558842105262
mean 1.9580025315789473


|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|
|--|--|--|--|--|--|--|--|--|--|
|3dunet|0.8582|0.6996|0.7789|21.05|25.95|23.5|2.53|2.11|2.32|
|unetr_48|0.8443|0.6929|0.7686|22.46|26.8|24.63|3.16|2.48|2.82|
|swinunetr_48| 0.8780|0.7555|0.8168|17.79|22.69|20.24|2.26|1.66|1.96|
|nnformer|0.8473|0.6876|0.7675|18.54|23.27|20.9|2.77|2.32|2.55|
|UXNET|0.8794|0.7686|0.824|17.97|22.42|20.2|2.15|1.67|1.91|
|mixunetr_dice_2222|0.8837|0.7739|0.8288|17.93|21.93|19.93|2.04|1.66|1.85|
|mixunetr_bdouloss_2222|0.8833|0.7792|0.8312|16.84|20.58|18.71|2.04|1.67|1.86|


|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|swinunetr_48|0.865|0.7999|0.8324|19.83|19.66|19.74|2.35|2.03|2.19|/home/data1/skyous/prostatex_crop_log/swinunetr/anatomyprostatex_crop20240321_1038_swinunetr_48_1/output|
|UXNET|0.86|0.7983|0.8292|19.95|18.21|19.08|2.31|2.05|2.18|./prostatex_crop_log/UXNET/anatomyprostatex_crop20240321_1043_UXNET_1/output|
|nnformer|0.8366|0.7341|0.7853|19.55|22.05|20.8|2.86|3.32|3.09|./prostatex_crop_log/nnformer/anatomyprostatex_crop20240321_1047_nnformer_1/output|
|mixunetr_dice|0.8669|0.7987|0.8328|19.45|18.99|19.22|2.24|2.01|2.13|/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1053_mixunetr_1/output|
|mixunetr_bdou|0.8611|0.7983|0.8297|19.77|18.84|19.31|2.49|2.1|2.29|/home/data1/skyous/prostatex_crop_log/mixunetr/anatomyprostatex_crop20240321_1057_mixunetr_1/output|
|3dResUNet|0.865|0.7918|0.8284|17.55|18.94|18.25|2.14|1.99|2.06|/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1132_3dunet_1/output|
|3dUNet|0.8541|0.7724|0.8132|18.98|18.41|18.7|2.61|2.24|2.43|/home/data1/skyous/prostatex_crop_log/3dunet/anatomyprostatex_crop20240321_1136_3dunet_1/output|
|unetr_42|0.8586|0.7873|0.823|19.93|18.72|19.32|2.61|2.11|2.36|/home/data1/skyous/prostatex_crop_log/unetr/anatomyprostatex_crop20240321_2041_unetr_1/output|